In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
# Set random seed for reproducibility
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
# Load dataset
dataset_path = r"C:\Users\HOME\OneDrive\Documents\Downloads\Auto_MPG_data.csv"
dataset = pd.read_csv(dataset_path)

# Preprocess data
X = dataset.drop(columns='MPG').values
y = dataset['MPG'].values
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=val_size, random_state=random_state, shuffle=is_shuffle
)
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=test_size, random_state=random_state, shuffle=is_shuffle
)

# Normalize data
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [5]:
# Build Dataset and DataLoader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [6]:
# Define the MLP Model
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        out = self.output(x)
        return out.squeeze(1)

input_dims = X_train.shape[1]
hidden_dims = 64
output_dims = 1
model = MLP(input_dims, hidden_dims, output_dims).to(device)


In [7]:
# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)


In [8]:
# Define R2 score function
def r_squared(y_true, y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2


In [9]:
# Train the model
epochs = 100
train_losses, val_losses, train_r2, val_r2 = [], [], [], []

for epoch in range(epochs):
    model.train()
    train_loss, train_target, train_predict = 0.0, [], []
    for X_samples, y_samples in train_loader:
        X_samples, y_samples = X_samples.to(device), y_samples.to(device)
        optimizer.zero_grad()
        outputs = model(X_samples)
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))
    
    model.eval()
    val_loss, val_target, val_predict = 0.0, [], []
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples, y_samples = X_samples.to(device), y_samples.to(device)
            outputs = model(X_samples)
            val_loss += criterion(outputs, y_samples).item()
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))

    print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')


Epoch 1/100 | Train Loss: 282.7696 | Val Loss: 88.6699
Epoch 2/100 | Train Loss: 137.6303 | Val Loss: 72.2912
Epoch 3/100 | Train Loss: 71.2735 | Val Loss: 19.7733
Epoch 4/100 | Train Loss: 24.1454 | Val Loss: 161.7169
Epoch 5/100 | Train Loss: 92.8138 | Val Loss: 21.3591
Epoch 6/100 | Train Loss: 17.4018 | Val Loss: 13.7650
Epoch 7/100 | Train Loss: 20.4737 | Val Loss: 12.0070
Epoch 8/100 | Train Loss: 37.7618 | Val Loss: 43.9195
Epoch 9/100 | Train Loss: 17.9690 | Val Loss: 39.0093
Epoch 10/100 | Train Loss: 23.7735 | Val Loss: 46.1633
Epoch 11/100 | Train Loss: 30.4893 | Val Loss: 20.4989
Epoch 12/100 | Train Loss: 9.9094 | Val Loss: 5.8080
Epoch 13/100 | Train Loss: 15.8280 | Val Loss: 14.4286
Epoch 14/100 | Train Loss: 12.0749 | Val Loss: 13.2556
Epoch 15/100 | Train Loss: 14.7775 | Val Loss: 8.1995
Epoch 16/100 | Train Loss: 10.7564 | Val Loss: 20.1989
Epoch 17/100 | Train Loss: 12.3653 | Val Loss: 14.8891
Epoch 18/100 | Train Loss: 15.5615 | Val Loss: 11.8103
Epoch 19/100 | Trai

In [ ]:
# Evaluate the model on test data

model.eval()
with torch.no_grad():
    y_hat = model(X_test.to(device))
    test_r2 = r_squared(y_test, y_hat) 
    print(f'Test R2 Score: {test_r2}')


Test R2 Score: 0.8778729438781738
